In [227]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

train_set_orig = datasets.FashionMNIST('MNIST_data/', download = True, train = True,  transform = transform)
valid_set_orig = datasets.FashionMNIST('MNIST_data/', download = True, train = False, transform = transform)

train_set_autoencoder = CustomDataset(train_set_orig)
valid_set_autoencoder = CustomDataset(valid_set_orig)

In [228]:
train_set_autoencoder = CustomDataset(train_set_orig)
valid_set_autoencoder = CustomDataset(valid_set_orig)

In [229]:
train_loader = DataLoader(train_set_autoencoder, batch_size=100, shuffle=True, num_workers=os.cpu_count()-1)
valid_loader = DataLoader(valid_set_autoencoder, batch_size=100, shuffle=False, num_workers=os.cpu_count()-1)

In [244]:
import os
import json
import torch
from autoencoder import Autoencoder, Autoencoder_no_lineal

# Ruta a la carpeta de resultados
results_folder = './results/'
files = os.listdir(results_folder)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = {}

json_files = [file for file in files if file.endswith('.json')]
model_files = [file for file in files if file.endswith('.pt')]

for file in json_files:
    with open(os.path.join(results_folder, file), 'r') as f:
        result = json.load(f)
        config_id = result['config']['id']
        model_path = os.path.join(results_folder, f"model_{config_id}.pt")
        
        if result['config'].get('lineal', True):
            dropout = result['config']['dropout']
            l_size = result['config']['l_size']
            model = Autoencoder(dropout=dropout, l_size=l_size)
        else:
            model = Autoencoder_no_lineal(dropout=result['config']['dropout'])
        
        # Cargar el estado del modelo
        aux = torch.load(model_path, map_location=device)
        
        try:
            model.load_state_dict(aux)
        except RuntimeError as e:
            print(f"Error al cargar el modelo para la configuración {config_id}: {e}")

            # dimensiones de aux
            print(aux.keys())
            # dimensiones de model
            print(model.state_dict().keys())
            continue
        
        models[config_id] = {
            'config': result['config'],
            'model': model
        }

print("Modelos cargados correctamente:", models.keys())

Error al cargar el modelo para la configuración 4: Error(s) in loading state_dict for Autoencoder:
	Missing key(s) in state_dict: "encoder.3.weight", "encoder.3.bias", "encoder.8.weight", "encoder.8.bias", "decoder.3.weight", "decoder.3.bias", "decoder.5.weight", "decoder.5.bias". 
	Unexpected key(s) in state_dict: "encoder.4.weight", "encoder.4.bias", "encoder.9.weight", "encoder.9.bias", "decoder.7.weight", "decoder.7.bias", "decoder.4.weight", "decoder.4.bias". 
	size mismatch for encoder.0.weight: copying a param with shape torch.Size([32, 1, 3, 3]) from checkpoint, the shape in current model is torch.Size([16, 1, 3, 3]).
	size mismatch for encoder.0.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for decoder.0.weight: copying a param with shape torch.Size([3136, 1024]) from checkpoint, the shape in current model is torch.Size([4608, 1024]).
	size mismatch for decoder.0.bias: copying a param with shap

/tmp/ipykernel_133673/1043210127.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aux = torch.load(model_path, map_location=device)
